In [35]:
import requests
import pandas as pd
# Importa bibliotecas para fazer requisições HTTP e manipular dados em tabelas
# `requests` acessa a API do Bacen; `pandas` organiza os dados em DataFrames

# Dicionário com os códigos SGS e suas descrições
series_sgs = {
    24363: ('Modalidade', 'Crédito pessoal'),
    24364: ('Modalidade', 'Aquisição de veículos'),
    24365: ('Modalidade', 'Crédito consignado'),
    20786: ('Modalidade', 'Cartão de crédito rotativo'),
    20787: ('Modalidade', 'Cartão de crédito parcelado'),
    24367: ('Modalidade', 'Cheque especial'),
    20611: ('Modalidade', 'Crédito habitacional'),
    24369: ('Setor', 'Pessoa física'),
    24370: ('Setor', 'Pessoa jurídica'),
}
# Define os códigos das séries temporais do Bacen e associa cada um a uma categoria e tipo
# Esses códigos serão usados para buscar dados específicos via API

def baixar_serie_sgs(codigo_serie, inicio='01/01/2015', fim='01/01/2025'):
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={inicio}&dataFinal={fim}'
    # Monta a URL da API do Bacen com o código da série e intervalo de datas
    # O formato JSON facilita a conversão para DataFrame

    try:
        response = requests.get(url)
        response.raise_for_status()  # Garante que o status é 200
        dados = response.json()
        # Faz a requisição à API e converte a resposta para JSON
        # Se houver erro de conexão ou status inválido, será tratado abaixo

        if not dados:
            print(f"⚠️ Série {codigo_serie} retornou vazia.")
            return pd.DataFrame()  # Retorna DataFrame vazio
        # Verifica se a série está vazia e evita erros futuros
        # Retorna um DataFrame vazio para manter consistência

        df = pd.DataFrame(dados)
        df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        categoria, tipo = series_sgs[codigo_serie]
        df['categoria'] = categoria
        df['tipo'] = tipo
        return df[['data', 'categoria', 'tipo', 'valor']]
        # Converte os dados para DataFrame e ajusta os tipos de dados
        # Adiciona colunas de contexto (categoria e tipo) e retorna colunas organizadas

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro na requisição da série {codigo_serie}: {e}")
        return pd.DataFrame()
    except ValueError as e:
        print(f"❌ Erro ao decodificar JSON da série {codigo_serie}: {e}")
        print("Resposta recebida:", response.text)
        return pd.DataFrame()
    # Trata erros de conexão ou de formatação dos dados
    # Garante que o código não quebre e sempre retorne um DataFrame válido

# Baixar todas as séries e concatenar
dfs = [baixar_serie_sgs(codigo) for codigo in series_sgs]
base_final = pd.concat(dfs, ignore_index=True).sort_values(by='data').reset_index(drop=True)
# Chama a função para cada código SGS e junta todos os DataFrames
# Ordena os dados por data e reorganiza os índices

# Exibir quantidade
print(f"Quantidade de linhas: {len(base_final)}")

# Exibir amostra
print(base_final.head())
# Mostra as primeiras 5 linhas da base final para conferência

# (Opcional) Exportar para CSV
# base_final.to_csv('inadimplencia_bacen.csv', index=False)
# Salva os dados em um arquivo CSV, útil para análise externa ou visualização no Excel

# # # # # # # # # # # # # # # # # exportacao# # # # # # # # # # # # # # # # # # # # 


# Exporta o arquivo CSV para o diretório especificado
from datetime import datetime

# Gera a data atual no formato YYYYmmdd
data_processamento = datetime.today().strftime('%Y%m%d')

# Monta o caminho completo com a data no nome do arquivo
caminho_arquivo = fr'C:\Users\diego\OneDrive\DATAMASTER\dados_bacen_new\inadimplencia_bacen_{data_processamento}.csv'

# Exporta o arquivo CSV
base_final.to_csv(caminho_arquivo, index=False, encoding='utf-8-sig')
print(f"✅ Arquivo exportado com sucesso: {caminho_arquivo}")






Quantidade de linhas: 933
        data   categoria                         tipo         valor
0 2015-01-01  Modalidade              Crédito pessoal      97.40111
1 2015-01-01  Modalidade         Crédito habitacional  377360.00000
2 2015-01-01  Modalidade  Cartão de crédito parcelado      13.64000
3 2015-01-01  Modalidade        Aquisição de veículos     101.65065
4 2015-01-01  Modalidade   Cartão de crédito rotativo      26.54000
✅ Arquivo exportado com sucesso: C:\Users\diego\OneDrive\DATAMASTER\dados_bacen_new\inadimplencia_bacen_20250815.csv
